In [1]:
import pandas as pd
import numpy as np

In [53]:
class NetPremium:
    
    def __init__(self, rate, i, SIZE=101, l0=10000):
        self.v = 1/(1+i)
        self.l0 = l0
        self.q_x = rate['mortality'].groupby('구분').apply(lambda df: df[['연령', '위험률']].sort_values(by='연령')['위험률'].to_numpy())
        self.q_w = rate['lapse'].sort_values(by='시점')['해지율'].to_numpy()
        
    def get_product(self):
        """상품명 리턴"""
        
        return '참좋은간편건강보험1904'
        
    def get_coverage(self):
        """담보명 리턴"""
        
        return '질병사망(간편고지)(갱신형, 비갱신형)'
    
    def __calc_maintainer(self, x, n, options):
        """유지자(순보험료 분자 부분) 계산"""
        
        # lx
        l1x = np.zeros(n+1)
        l2x = np.zeros(n+1)
        l3x = np.zeros(n+1)
        l4x = np.zeros(n+1)
        l1x[0] = self.l0
        l2x[0] = self.l0
        l3x[0] = self.l0
        l4x[0] = self.l0
        for t in range(n):
            l1x[t+1] = l1x[t]*(1-self.q_x['제자리암(상피내암)발생률'][x+t])*(1-self.q_w[t])
            l2x[t+1] = l2x[t]*(1-self.q_x['경계성종양발생률'][x+t])*(1-self.q_w[t])
            l3x[t+1] = l3x[t]*(1-self.q_x['기타피부암발생률'][x+t])*(1-self.q_w[t])
            l4x[t+1] = l4x[t]*(1-self.q_x['갑상선암발생률'][x+t])*(1-self.q_w[t])
            
        # Cx, Mx
        C1x = l1x*self.q_x['제자리암(상피내암)발생률'][x:(x+n+1)]*self.v**(np.arange(n+1)+0.5)*(1-self.q_w[:(n+1)]/2)
        C2x = l2x*self.q_x['경계성종양발생률'][x:(x+n+1)]*self.v**(np.arange(n+1)+0.5)*(1-self.q_w[:(n+1)]/2)
        C3x = l3x*self.q_x['기타피부암발생률'][x:(x+n+1)]*self.v**(np.arange(n+1)+0.5)*(1-self.q_w[:(n+1)]/2)
        C4x = l4x*self.q_x['갑상선암발생률'][x:(x+n+1)]*self.v**(np.arange(n+1)+0.5)*(1-self.q_w[:(n+1)]/2)
        M1x = C1x[::-1].cumsum()[::-1]
        M2x = C2x[::-1].cumsum()[::-1]
        M3x = C3x[::-1].cumsum()[::-1]
        M4x = C4x[::-1].cumsum()[::-1]

        
        if ((options['new'] and ~options['renewal']) or options['renewal']):
            return 0.5*(3/4)*(C1x[0]+C2x[0]+C3x[0]+C4x[0]) \
                + (M1x[0]-M1x[n]) + (M2x[0]-M2x[n]) + (M3x[0]-M3x[n]) + (M4x[0]-M4x[n])
        else:
            return (M1x[n]-M1x[n]) + (M2x[0]-M2x[n]) + (M3x[0]-M3x[n]) + (M4x[0]-M4x[n])
    
    def __calc_payer(self, x, m, mpp, options):
        """납입자(순보험료 분모 부분) 계산"""
        
        # 1종, 3종, 9종
        if options['types'] in ['1종', '3종', '9종']:
            q_j = list(map(lambda k: self.q_x[k], ['간편고지상해1급80%이상후유장해발생률', '간편고지질병80%이상후유장해발생률', '기타피부암및갑상선암이외의암발생률', '뇌졸중발생률', '급성심근경색증발생률']))
            
            # lpx
            lpx = np.zeros(m+1)
            lpx[0] = self.l0
            for t in range(m):
                lpx[t+1] = lpx[t]*(1-self.q_w[t])
                for j in range(len(q_j)):
                    Dj = 3/4 if j==2 and ((options['new'] and ~options['renewal']) or options['renewal']) and t == x else 1
                    lpx[t+1] *= 1-Dj*q_j[j][t]

            # Dpx, Npx
            Dpx = lpx*self.v**np.arange(m+1)
            Npx = Dpx[::-1].cumsum()[::-1]

        # 5종, 7종, 11종
        elif options['types'] in ['5종', '7종', '11종']:

            # lpx
            lpx = np.zeros(m+1)
            lpx[0] = self.l0
            for t in range(m):
                lpx[t+1] = lpx[t]*(1-self.q_w[t])

            # Dpx, Npx
            Dpx = lpx*self.v**np.arange(m+1)
            Npx = Dpx[::-1].cumsum()[::-1]

        else:
            raise Exception('종구분오류')
            
        return mpp*((Npx[0]-Npx[m])-(mpp-1)/(2*mpp)*(Dpx[0]-Dpx[m]))
    
    def calc_net_premium(self, x, n, m, mpp, options):
        """순보험료 계산"""
        
        return self.__calc_maintainer(x, n, options)/self.__calc_payer(x, m, mpp, options)

In [54]:
# 데이터 불러오기
mortality_rate = pd.read_excel('data/참좋은간편건강보험1904_기초데이터_20200805.xlsx', sheet_name='위험률', dtype={'연령': int, '위험률': float})
lapse_rate = pd.read_excel('data/참좋은간편건강보험1904_기초데이터_20200805.xlsx', sheet_name='해지율', dtype={'시점': int, '해지율': float})
rate = {'mortality': mortality_rate, 'lapse': lapse_rate}

In [60]:
# 테스트
net_premium = NetPremium(rate, 0.02)
x, n, m, mpp = 30, 20, 10, 12
100000000*net_premium.calc_net_premium(x, n, m, mpp, {'types': '5종', 'renewal': True, 'new': True})

33252.19307880745